We start by removing all trailing/leading whitespaces, and by ensuring that all data is decoded corectly using the mosdle ftfy

In [1]:
import pandas as pd
import ftfy
import re
# We consider all data as strings . We use python as engine due to the messines of the data . We use quotechar because i've saw in the dataset we have some texts that use "", in order to 'escape' the comma limitation
df = pd.read_csv("facebook_dataset.csv", engine="python", quotechar='"', on_bad_lines='skip', dtype=str)
df.fillna('', inplace=True)

# Remove leading and trailing whitespaces using regex
df = df.map(lambda x: re.sub(r'\s+', ' ', x).strip() if isinstance(x, str) else x)

# Ensure all elements from DataFrame are decoded correctly
df = df.map(lambda x: ftfy.fix_text(x) if isinstance(x, str) else x)
df.to_csv("facebook.csv",encoding="utf-8-sig",index=False)




We continue by completing the country name , and country code . If we know the code , then we know the country . If we know the country , then we know the code. But in order for us to do that , we must first scrape data about countries from a website . We are interested in the country name, Iso-2 country code, and country code phone number:

country_code: ->it could be empty. But if we did have the name of the country, we could easily find its country code using the scraped data
            ex:(suppose the country_name is france)
                Before:""
                After :fr 

country_name: ->it could be empty. But if we did have the code of the country, we could easily find its country name using the scraped data
            ex:(suppose the country_code is fr)
                Before:""
                After :france
For PuertoRico and --------------------- , we need to delete some of the country phone code options in order to keep a clean csv file (these countries have more than 1 country phone code)

In [2]:
#in order to assure that the phone numbers are listed corectly (they start with the right digits) 
# we scrap some data from a site 
#Here we get :
# ->The Name of the country
# ->The digit combination for phone number for each country (parsed)
# ->The 2-ISO code of each country (parsed)


from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

if os.path.exists("countryCodes.csv"):
    with open("countryCodes.csv", 'w') as file:
        pass  # This will clear the file content
    print(f"The file '{"countryCodes.csv"}' has been cleared.")


def parseIsoCodes(code):
    return code.split(' / ')[0][:2]

def parseCountryCode(country_code):
    return country_code.replace('-', '')

# Fetch the data from the website
response = requests.get("https://countrycode.org/")
soup = BeautifulSoup(response.content, 'html.parser')

# Find all rows in the table body
rows = soup.find('tbody').find_all('tr')
fileName = "countryCodes.csv"

# Write header for CSV file and clear previous content
with open(fileName, 'w') as file:
    file.write("Name,Code,ISO\n")

# Scrape data and write to the CSV file
for row in rows:
    country_name = row.find('a').text
    country_code = row.find_all('td')[1].text
    iso_codes = row.find_all('td')[2].text
    with open(fileName, 'a') as file:
        file.write(country_name.lower() + "," + parseCountryCode(country_code).lower() + "," + parseIsoCodes(iso_codes).lower() + "\n")

# Function to replace a specific row in the file
def replace_row_in_file(row_number, new_content):
    # Read all lines from the file
    with open(fileName, 'r') as file:
        lines = file.readlines()
    
    # Check if the row_number is within the range of the file
    if 0 <= row_number < len(lines):
        # Replace the specific row
        lines[row_number] = new_content + '\n'
    
    # Write the modified lines back to the file
    with open(fileName, 'w') as file:
        file.writelines(lines)

replace_row_in_file(60, "dominican republic,1809,do")
replace_row_in_file(171, "puerto rico,1787,pr")

countries=pd.read_csv("countryCodes.csv",on_bad_lines="warn",dtype=str,quotechar='"')




The file 'countryCodes.csv' has been cleared.


Now we have the data about countries thanks to "https://countrycode.org/" . We complete the missing data as explained before. There may be some problems with this aproach since im suppose to merge(maybe the original data had russia and the countryCodes.csv has russian federation , but we'll solve this later) .

In [3]:


data = pd.read_csv("facebook.csv", quotechar='"', on_bad_lines='warn', engine='python',dtype=str)

data['country_name'] = data['country_name'].str.lower()
data['country_code'] = data['country_code'].str.lower()

# Process each record
for index, record in data.iterrows():
    country_name = record["country_name"].strip() if isinstance(record["country_name"], str) else ''
    if country_name != '':
        country_code = countries[countries['Name'] == country_name]['ISO']
        if not country_code.empty:
            data.at[index, "country_code"] = country_code.values[0]
    country_code = record["country_code"].strip() if isinstance(record["country_code"], str) else ''
    if country_code != '':
        country_name = countries[countries['ISO'] == country_code]['Name']
        if not country_name.empty:
            data.at[index, "country_name"] = country_name.values[0]

    
data.to_csv("facebook.csv", index=False, encoding='utf-8-sig')


Now we need to format the phone number .We need to take in conisderation the case when the number is printed as a scientific notation , the case when the number of characters is diffrent than 12 and the case wwhen the phone number doesnt start with the usual characters for the country from which the phone number is posted.

In [4]:

data = pd.read_csv("facebook.csv", quotechar='"', on_bad_lines='warn', engine='python',dtype=str)


def is_scientific_notation(value):
    pattern = r'^\d+\.?\d*E[+-]?\d+$'
    return bool(re.match(pattern, value))

def format_phone_number(sci_notation):
    phone_number = float(sci_notation)
    phone_str = str(int(phone_number))
    if len(phone_str) >= 10:
        return str(f"{phone_str[:3]}{phone_str[3:6]}{phone_str[6:]}")
    else:
        return "Invalid phone number length"


nrOfWrongNumbers=0

def transformDataFrameIntoArray(dataframe):
    value = []
    for i in range(len(dataframe)):
        value.extend(dataframe.iloc[i].tolist())  # Append values one by one
    return value

for index,record in data.iterrows():
    recordPhone=str(record['phone'])
    recordCountry=str(record['country_name'])
    phoneCodeCountry=transformDataFrameIntoArray(countries[countries["Name"]==recordCountry])[1] if countries[countries["Name"]==recordCountry].empty==False else ""
    # print(recordPhone+"     "+phoneCodeCountry+"     "+recordCountry)
    if(is_scientific_notation(recordPhone)):
        recordPhone="+"+format_phone_number(recordPhone)
        data.at[index,"phone"]=recordPhone
    if(recordPhone!="nan" and recordPhone!=""):
        if(recordPhone[1:].startswith(str(phoneCodeCountry))==False or len(recordPhone)!=12):
            nrOfWrongNumbers=nrOfWrongNumbers+1  
            data.at[index,"phone"]="WRONG"

print(f"we have a total of {nrOfWrongNumbers} wrong numbers")
data.to_csv("facebook.csv", index=False,encoding="utf-8-sig")







we have a total of 1651 wrong numbers


Now we will erase the unnecesary data from the adress column. In many cases the adress column contains data from other columns asweel , like zipcode , country name , country code and so on...
        ex:
            Before :134 rue entrepreneurs, za du vigné, 30420, calvisson, france, languedoc-roussillon
            After  :134 rue entrepreneurs,za du vigné,languedoc-roussillon

In [5]:
import pandas as pd

# Read CSV in chunks
chunk_size = 1000  # Adjust based on your system's capabilities
chunks = pd.read_csv("facebook.csv", dtype=str, engine="python", quotechar='"', on_bad_lines="warn", chunksize=chunk_size)

def transformArrayintoString(array):
    value=""
    for elem in array:
        value=value+elem+","
    return value[:-1]

# Process each chunk
for chunk in chunks:
    chunk = chunk.fillna('')
    for index, record in chunk.iterrows():
        adresaDetailArrayInitial = record["address"].split(",")
        adresaDetailArrayNew = []
        for elem in adresaDetailArrayInitial:
            elem = elem.strip()
            exista = False
            for column in chunk.columns:
                if column != "address":
                    if chunk.loc[index, column].strip() == elem:
                        exista = True
            if not exista:
                adresaDetailArrayNew.append(elem)
                
        chunk.at[index, "address"] = transformArrayintoString(adresaDetailArrayNew) if len(adresaDetailArrayNew) != 0 else chunk.at[index, "address"]

    # Write the processed chunk to a new CSV file without the index
    chunk.to_csv("facebookCHUNKY.csv", mode='a', header=not (chunk.index[0] > 0), encoding="utf-8-sig", index=False)
chunkyData=pd.read_csv("facebookCHUNKY.csv",on_bad_lines="warn",engine="python",quotechar='"',dtype=str)
chunkyData.to_csv("facebook.csv",encoding="utf-8-sig",quotechar='"',mode='w',index=False)
os.remove(path="facebookCHUNKY.csv")


Now we will go ahead and check all the emails, using the nastiest regex i ever saw in my life . If the email doesnt have the correct form , we change it to "WRONG".

In [6]:

df = pd.read_csv("facebook.csv", engine="python", quotechar='"', on_bad_lines="warn",dtype=str)
pattern = r"(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|\"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*\")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\\])"
nrOfWrongEmails=0
# Check each email
for index, record in df.iterrows():
    email = str(record["email"])  # Convert to string
    if not re.match(pattern, email) and email!="nan" and email!="":
        df.at[index,"email"]="WRONG"
        nrOfWrongEmails=nrOfWrongEmails+1
print(f"We have a total of {nrOfWrongEmails} incorrectly formated emails")
df.to_csv("facebook.csv",encoding="utf-8-sig",index=False)


       


We have a total of 26 incorrectly formated emails


Now we will check the links, using a regex. If the link doesnt have the correct form we change it to "WRONG".

In [7]:

df = pd.read_csv("facebook.csv", engine="python", quotechar='"', on_bad_lines="warn",dtype=str)
pattern="^https?:\\/\\/(?:www\\.)?[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$"
nrOfWrongLinks=0
for index,record in df.iterrows():
    link=record["link"]
    if not re.match(pattern,link) and link!="nan" and link!="":
        df.at[index,"link"]="WRONG"
        nrOfWrongLinks=nrOfWrongLinks+1
print(f"We have a total of {nrOfWrongLinks} incorrectly formated links")
df.to_csv("facebook.csv",encoding="utf-8-sig",index=False)




We have a total of 133 incorrectly formated links


This is not a smart approach, we should've drop duplicates from the begging , not at the end, but we learn . (Luckily there are no duplicates , so it doesnt affect us)

In [8]:

df = pd.read_csv("facebook.csv", dtype=str, engine="python", quotechar='"', on_bad_lines="warn")

# Check for duplicate rows
# `subset=None` checks all columns; you can specify column names if needed
duplicates = df[df.duplicated(subset=None)]

# Print the duplicates
if not duplicates.empty:
    print(f"Found {len(duplicates)} duplicate rows:\n")
    print(duplicates)
else:
    print("No duplicates found.")




No duplicates found.


I have tried to fill in the values of zipcodes for the records of which we know the city , but it is a mess, the alrgorithm takes way to long, and is unale to finish ,and some result differ than the ones from the dataset

In [9]:
if False:
    from geopy.geocoders import Nominatim

    geolocator = Nominatim(user_agent="facebook_cleaner")

    # location=geolocator.geocode("calvisson",addressdetails=True)
    # print(location.raw["address"]["postcode"])
    df=pd.read_csv("facebook.csv",dtype=str,quotechar='"',engine="python",on_bad_lines="warn")
    df.fillna('',inplace=True)
    uniqueCities=df["city"].unique()
    with open("codesVsActualCode.txt", 'w') as file:
        for city in uniqueCities:
            try:
                cityZipCodeGEO=geolocator.geocode(city,addressdetails=True).raw["address"]["postcode"]  
                file.writelines(f"The name of the city is {city};using geopy we get {cityZipCodeGEO}\n")
            except KeyError as e:
                print(e)
            except AttributeError as e2:
                print(e2)
          
    

However , seeing the shape of the address column there must be a way to actually do this ( fill in region , region code , zip code ...) . We start with what seems to look like a city name
example:2989 unity road,k0h 1m0,kingston,on,ontario
2989 unity road -address
k0h 1m0 -zipcode
on - region code
ontario -region name
kingston -city
WE WILL FIND A WAY !

We did indeed found a way . however , if a certain zipcode is both zipcode and region name we may have some problems . 

In [12]:

import pycountry


def is_region_name(regionName, countryCode):
    for region in pycountry.subdivisions.get(country_code=countryCode):
        if regionName.lower() == region.name.lower():
            return True
    return False

def is_zip_code(zipCode, countryCode):
    zipCode = zipCode.upper()
    countryCode = countryCode.upper()
    realZipCodePattern = zipPatterns[zipPatterns['Code'] == countryCode]['Pattern']    
    if not realZipCodePattern.empty:
        pattern = realZipCodePattern.iloc[0]  # Get the first pattern as a string
        print(f"Pattern for {countryCode}: {pattern}")
        return re.match(pattern, zipCode) is not None  # Return True or False
    return False  # Return False if no pattern is found

def is_country_code(countryCode):
    countryCode=countryCode.upper()
    if not zipPatterns[zipPatterns['Code'] == countryCode].empty:
        return True
    return False
#we use that strange character to make sure \ is not an escape character, and since we are working wth regexes , many characters are already "in use"
zipPatterns = pd.read_csv("regexZipCodesForCountries.csv", engine="python", dtype=str, on_bad_lines="warn", quotechar='"', escapechar='௸')
# print(is_country_code("ra")) 




False
